In [1]:
import os
import pymol
from pymol import cmd
import numpy as np
import __main__
import pandas as pd
import sys

In [2]:
def modify_pdb(path, length):
    #this is for protein data segment not conformer.
    name = path.replace('.pdb','')
    
    #add .pdb to just test here
    path = 'loop_target/' + path 
    length = length
    dir_path = 'loop_target/' + name + '_' + str(length)
    
    if os.path.isdir(dir_path) == False:
        os.mkdir(dir_path)
        
    file = open(path, 'r')
    txt = file.readlines()
    
    
    res_start = int(txt[0][23:26])
    res_end = int(txt[-2][23:26])

    
    distance = res_end - res_start + 1
    change = distance - length + 1
    #change ex:AAAAAAAAAA 10 sub sgments of 6 
    #this renames them 
    for i in range(change):
        start = res_start + i
        output_name = dir_path + '/' + str(start) + '_' + str((start+length)) + '.pdb'
        output = open(output_name, 'w')
        for j in range(len(txt)):
            number = txt[j][23:26].replace(' ','')
            if number.isnumeric():
                res_id = int(number)
                if txt[j][:4] == 'ATOM' and res_id >= start and res_id <= (start+length-1):
                    id = res_id - start + 1
                    if id < 10:
                        data = txt[j][:17] + 'UNK' + txt[j][20:23] + ' ' + str(id) + txt[j][26:]
                        output.write(data)
                    else:
                        data = txt[j][:17] + 'UNK' + txt[j][20:23] + str(id) + txt[j][26:]
                        output.write(data)
        output.write('END')
        output.close()

In [3]:
#modify_pdb("ligand",4)

In [4]:
def alignment(loop_name, n):
    #4 to 12 because 4 to 12 residue length.
    #use left ring only
    #change this to 1-10 for left ring
    
    #breaks up pdb protein file
    for i in range(1,11):
        pdb_name = loop_name + '.pdb'
        modify_pdb(pdb_name, i)
    __main__.pymol_argv = [ 'pymol', '-qc'] 
    #pymol.finish_launching()
    
    #directory containing all the General CLIPS folder, inside this folder is list of folder containing left ring sizes Ex:4 
    dir_1 = os.listdir('CLIPS_modified/2/')
    dir_1.sort()
    dir_path = 'align_result_CLIPS/' + loop_name
    
    if os.path.isdir(dir_path) == False:
        os.mkdir(dir_path)
    folderCounter = 1
    #for each folder in CLIPS modified Ex:1,2,3,4,5,6,7,8,9,10
    #change to 4 
    for ringLength in range (4 , 11):
        dir_2 = []
        for i in range(len(dir_1)):
            #folder path would be CLIPS_modified + 1,2,3,ect..
            name_1 = 'CLIPS_modified/2/' 
            folder = str(ringLength)
            
            file_listFirstRingLengthFolder = ""
            file_listSecondRingLengthFolder = ""
            reachedFirstBreakFolder = False
            AcounterFolder = 0
                        
            for l in dir_1[i]:
                if(not l.isdigit()):
                    if l == 'A':
                        AcounterFolder += 1
                        reachedFirstBreakFolder = True
                if(l.isdigit() and not reachedFirstBreakFolder):
                    file_listFirstRingLengthFolder += l
                if(l.isdigit() and reachedFirstBreakFolder and AcounterFolder != 2):
                    file_listSecondRingLengthFolder += l 
                
            file_listFirstRingLengthFolder  = int(file_listFirstRingLengthFolder)
            file_listSecondRingLengthFolder = int(file_listSecondRingLengthFolder) 
            
            #print(file_listSecondRingLengthFolder)
            if str(file_listSecondRingLengthFolder) == str(folder):
                dir_2.append(dir_1[i])
            dir_2.sort()


            output_name = []
            output_rmsd = []

            #loop name is entered in function so ligand
            loop_path = loop_name + '_' + str(int(folder))
        print(dir_2)
        for j in range(len(dir_2)):

            name_2 = name_1 + '/' + dir_2[j]
            dir_3 = os.listdir(name_2)
            dir_3.sort()

            file_name = []
            RMSD = []
            #for each .mol2 file 
            for k in range(len(dir_3)):
                file_listFirstRingLength = ""
                file_listSecondRingLength = ""
                reachedFirstBreak = False
                Acounter = 0
                        
                for c in dir_3[k]:
                    if(not c.isdigit()):
                        if c == 'A':
                            Acounter += 1
                            reachedFirstBreak = True
                    if(c.isdigit() and not reachedFirstBreak):
                        file_listFirstRingLength += c
                    if(c.isdigit() and reachedFirstBreak and Acounter != 2):
                        file_listSecondRingLength += c 
                
                file_listFirstRingLength  = int(file_listFirstRingLength)
                file_listSecondRingLength = int(file_listSecondRingLength) 
                
                #print("new finder 1: " + str(file_listFirstRingLength))
                #print("new finder 1: " + str(file_listSecondRingLength))
                
                try:
                    print(dir_3[k]) # each actual .mol2 file
                    name_3 = name_2 + '/' + dir_3[k] #directory of each mol2 file
                    #print(loop_path)
                    
                    loop_list = os.listdir('loop_target/'+loop_path)
                    for s in range(len(loop_list)):
                        #mobile is protein
                        cmd.load('loop_target/' + loop_path + '/' + loop_list[s],'mobile')
                        #print(loop_list[s])
                        #target is .mol2 files
                        cmd.load(name_3,'target')
                        cmd.remove('hydrogen')
                        
                        #rmsd = cmd.align('mobile', 'target', cycles=0)
                        #rmsd how good it alligns. smaller better
                        #need to change it to i. 1 - End of Left Ring
            
                        #left ring starts at 1
                        #if C = 19 then ring left 1-10 contained by 1-8 + 9,10,11
                        #next segment starts at (15 + 7 + 5*rightringlength + 6) - 3
                        #last segment (15 + 7 + 5*rightringlength + 6) - (15 + 7 + 5*rightringlength + 6) + 5(leftringlength) +3
                        
                        #if N = 19 then ring left 1-8 9,14,13
                        #15 - 18 and 19- 19 + leftringlength * 5 + 2
                        
                        #opening file to check what is contained at atom 19
                        bicyclic = open(name_3, "r")
                        ligandFile = open('loop_target/' + loop_path + '/' + loop_list[s], "r")
                        
                        text = bicyclic.readlines()
                        ligandText = ligandFile.readlines()
                        #print(text[26][8])
                        
                        if text[26][8] == 'N':
                            #contains N 19
                            #rmsd = cmd.align('mobile & i. 1-9', 'target & i. 1-9', cycles=0) + cmd.align('mobile & i. 13-14', 'target & i. 13-14', cycles=0)+ cmd.align('mobile & i. 15-' + str(file_listSecondRingLength * 5 + 2), 'target & i. 15-' + str(file_listSecondRingLength * 5 + 2), cycles=0)
                            #print(str(19 + file_listSecondRingLength * 5 + 2))
                            #rmsd = cmd.align('mobile & i. 1-' + str(19 + file_listSecondRingLength * 5 + 2), 'target & i. 1-' + str(19 + file_listSecondRingLength * 5 + 2), cycles=0)
                            endAtom = 19 + file_listSecondRingLength * 5 
                            #print(str(endAtom))
                            #rmsd = cmd.align('mobile & (i. 1-9 or i. 13-' + str(endAtom) + ')', 'target & (i. 1-9 or i. 13-' + str(endAtom) + ')', cycles=0)
                            
                            #change to only alanine
                            #change target bounds as in files
                            #meeting time: 11AM
                            
                            #rmsd = cmd.align('mobile & (i. 1-9 or i. 13-' + str(endAtom) + ')', 'target & (i. 1-' + str(endAtom - 3) + ')', cycles=0)
                            #print(endAtom)
                            #print(ligandText[0][9] + ligandText[0][10])
                            #print(ligandText[len(ligandText) - 2][9] + ligandText[len(ligandText) - 2][10])
                            #rmsd = cmd.align('mobile & i. 1-' + str(75), 'target & i. 20-' + str(endAtom), cycles=0)
                            
                            rmsd = cmd.align('mobile & i. 1-' + str(file_listSecondRingLength), 'target & i. 2-' + str(file_listSecondRingLength + 1), cycles=0)
                            #rmsd = cmd.align('mobile & i. 1-' + str(file_listSecondRingLength), 'target & i. 24-25', cycles=0)
                            #print('mobile & i. 1-' + str(file_listSecondRingLength) + "," + 'target & i. 2-' + str(file_listSecondRingLength + 1))
                        
                        else:
                            
                            #rmsd = cmd.align('mobile & (i. 1-11 or i. 12-13)', 'target & (i. 1-11 or i. 12-13)', cycles=0) 
                            #contains C 19 
                            middleAtom = 15 + 7 + file_listFirstRingLength *5 + 6 -3 + 5
                            lastAtom = middleAtom + 15 + 7 + 5*file_listFirstRingLength + 6 + 5 * file_listSecondRingLength +3
                            lastAtom = lastAtom - middleAtom - 2
                            #print(str(middleAtom) + " " + str(lastAtom))
                            #atomDiff = lastAtom - middleAtom + 1
                            #print( str(middleAtom) + " " + str(lastAtom)+ " " + str(atomDiff))
                            #rmsd = cmd.align('mobile & (i. 1-11 or i. ' + str(middleAtom) + '-' + str(lastAtom) + ')', 'target & (i. 1-11 or i. ' + str(middleAtom) + '-' + str(lastAtom) + ')', cycles=0) 
                            #rmsd = cmd.align('mobile & (i. 1-11 or i. ' + str(middleAtom) + '-' + str(lastAtom) + ')', 'target & (i. 1-' + str(11 + atomDiff) + ')', cycles=0) 
                            
                            rmsd = cmd.align('mobile & i. 1-' + str(file_listSecondRingLength), 'target & i. 2-' + str(file_listSecondRingLength + 1), cycles=0)
                            
                            #print('target & i. 2-' + str(file_listSecondRingLength + 1))

                        bicyclic.close()
                        ligandFile.close()
                        #rmsd = cmd.align('mobile & i. 1-' + str(int(folder)+2), 'target & i. 1-' + str(int(folder)+2), cycles=0)
                        
                        complex_pair = []
                        complex_pair.append(dir_3[k])
                        complex_pair.extend(loop_list[s])
                        
                        file_name.append(complex_pair)
                        RMSD.append(rmsd[0])
                        
                        cmd.delete('target')
                        cmd.delete('mobile')
                    cmd.reinitialize()
                except Exception as ex:
                    template = "An exception of type {0} occurred. Arguments:\n{1!r}"
                    message = template.format(type(ex).__name__, ex.args)
                    print(message)
                    print('{} is error'.format(dir_3[k]))
            index_min = np.argmin(RMSD)
            output_name.append(file_name[index_min])
            output_rmsd.append(RMSD[index_min])
        final_output_name = []
        final_output_rmsd = []
        #gets best 50 of everything 
        for i in range(n):
            print(output_rmsd)
            min_id = np.argmin(output_rmsd)
            final_output_name.append(output_name[min_id])
            final_output_rmsd.append(output_rmsd[min_id])
            output_name.remove(output_name[min_id])
            output_rmsd.remove(output_rmsd[min_id])
        df = pd.DataFrame(final_output_name, final_output_rmsd)
        df.to_csv(dir_path + '/' + folder + '_2' + '.csv')

In [5]:
alignment('ligand',1)

['6A-4A4']
6A-4A4_1.mol2
 PyMOL not running, entering library mode (experimental)
6A-4A4_1003.mol2
6A-4A4_1004.mol2
6A-4A4_1008.mol2
6A-4A4_1010.mol2
6A-4A4_1011.mol2
6A-4A4_1013.mol2
6A-4A4_1014.mol2
6A-4A4_1015.mol2
6A-4A4_1019.mol2
6A-4A4_1022.mol2
6A-4A4_1027.mol2
6A-4A4_1032.mol2
6A-4A4_1034.mol2
6A-4A4_1037.mol2
6A-4A4_104.mol2
6A-4A4_1040.mol2
6A-4A4_1042.mol2
6A-4A4_1044.mol2
6A-4A4_1046.mol2
6A-4A4_1047.mol2
6A-4A4_1049.mol2
6A-4A4_1051.mol2
6A-4A4_1054.mol2
6A-4A4_1055.mol2
6A-4A4_1056.mol2
6A-4A4_1058.mol2
6A-4A4_1059.mol2
6A-4A4_1062.mol2
6A-4A4_1063.mol2
6A-4A4_1069.mol2
6A-4A4_1071.mol2
6A-4A4_1073.mol2
6A-4A4_1075.mol2
6A-4A4_1076.mol2
6A-4A4_1077.mol2
6A-4A4_108.mol2
6A-4A4_1080.mol2
6A-4A4_1085.mol2
6A-4A4_1087.mol2
6A-4A4_1094.mol2
6A-4A4_1096.mol2
6A-4A4_1099.mol2
6A-4A4_1100.mol2
6A-4A4_1101.mol2
6A-4A4_1109.mol2
6A-4A4_1110.mol2
6A-4A4_1111.mol2
6A-4A4_1114.mol2
6A-4A4_1115.mol2
6A-4A4_1116.mol2
6A-4A4_1119.mol2
6A-4A4_1121.mol2
6A-4A4_1124.mol2
6A-4A4_1125.mol2
6A

KeyboardInterrupt: 

In [14]:
if __name__== "__main__":
    alignment(sys.argv[1],100)

FileNotFoundError: [Errno 2] No such file or directory: 'loop_target/-f.pdb'